<a href="https://colab.research.google.com/github/MinsuChoKW/Data-Analytics-2025/blob/main/week5_process_discovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Week 5 - 프로세스 발견 (Process Discovery) (PM4Py)
- 실습 항목: 프로세스 모델 도출, Quality 측정 등

In [ ]:
# ✅ 환경 점검: PM4Py 버전 확인 (필요 시 설치)
try:
    import pm4py
    print("pm4py version:", pm4py.__version__)
except Exception as e:
    print("pm4py가 설치되어 있지 않거나 환경 문제가 있습니다.")
    print("Colab/로컬에서 아래 주석을 해제해 설치하세요:")
    print("!pip install -U pm4py")

## 환경 설정 (Setup)

본 노트북에서는 아래의 라이브러리를 활용한다:

* [PM4Py](https://pm4py.fit.fraunhofer.de/)

## Process Discovery

### Process Models

프로세스 모델의 대표적인 표준 표기법은 [BPMN](https://www.bpmn.org/) 이며, [BPMN.io](https://bpmn.io/)와 같은 웹 기반 도구로 작성할 수 있고, PM4Py에서 로드하여 활용할 수 있다.

In [ ]:
import pm4py

bpmn_graph = pm4py.read_bpmn("../data/sepsis.bpmn")

In [ ]:
pm4py.vis.view_bpmn(bpmn_graph)

위 예시는 BPMN으로 표기된 패혈증 프로세스 모델의 한 예를 나타낸다.

### Quality Dimensions

프로세스 모델의 품질을 평가하는 주요 차원으로는 `fitness`, `precision`, `simplicity`, `generalisation` 이 있다. 본 사례에서는 Wil van der Aalst의 저서 `Process Mining - Data Science in Action`에서 사용된 예시 이벤트 로그(`../data/Lfull.xes`)를 사용한다.

In [ ]:
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.inductive import algorithm as im_discovery
from pm4py.algo.evaluation.replay_fitness import algorithm as fitness_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.algo.evaluation.simplicity import algorithm as simplicity_evaluator
from pm4py.algo.evaluation.generalization.variants.token_based import apply as generalization_evaluator_apply
from pm4py.objects.conversion.process_tree import converter as pt_converter

example_log = xes_importer.apply('../data/Lfull.xes')

본 이벤트 로그 내 Activity 라벨은 아래와 같이 맵핑된다.

* a = register request,
* b = examine thoroughly,
* c = examine casually,
* d = check ticket,
* e = decide,
* f = reinitiate request,
* g = pay compensation, and
* h = reject request.

In [ ]:
pm4py.convert_to_dataframe(example_log)

DFG 도구를 통해 프로세스 모델을 도출하면 아래와 같다.

In [ ]:
pm4py.view_dfg(*pm4py.discover_directly_follows_graph(example_log))

Inductive Mining을 통해 프로세스 모델을 도출하면 아래와 같다.

In [ ]:
process_tree = im_discovery.apply(example_log)
pm4py.view_process_tree(process_tree)

Inductive Mining을 통해 도출된 Process Tree를 페트리넷으로 변환하면 아래와 같다.

In [ ]:
net, initial_marking, final_marking = pt_converter.apply(process_tree)
pm4py.view_petri_net(net, initial_marking, final_marking)

도출된 페트리 넷을 바탕으로 4개의 품질 측정 도구를 통해 결과를 도출한다.

In [ ]:
# Measure Fitness
# The 'token_replay' method is used for fitness evaluation
fitness = fitness_evaluator.apply(log, net, initial_marking, final_marking)
print(f"Fitness: {fitness['log_fitness']:.4f}")

# Measure Precision
# 'log_fitness' is used to evaluate the model
precision = precision_evaluator.apply(log, net, initial_marking, final_marking)
print(f"Precision: {precision:.4f}")

# Measure Simplicity
simplicity = simplicity_evaluator.apply(net)
print(f"Simplicity: {simplicity:.4f}")

# Measure Generalization
generalization = generalization_evaluator_apply(log, net, initial_marking, final_marking)
print(f"Generalization: {generalization:.4f}")